In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Dense, Dropout, Lambda, multiply
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import os

In [ ]:
# Load data
base_image_dir = os.path.join('.', 'C:/Users/Sarang/Documents/Final Year Project/train/train')
df = pd.read_csv(os.path.join('C:/Users/Sarang/Documents/Final Year Project/train/trainLabels.csv'))
df['path'] = df['image'].map(lambda x: os.path.join(base_image_dir, '{}.jpeg'.format(x)))
df['exists'] = df['path'].map(os.path.exists)
df = df[df['exists']]
df = df.drop(columns=['image', 'exists'])
df['level'] = df['level'].astype(str)

In [ ]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255
)

x_train = train_datagen.flow_from_dataframe(
    train_df,
    directory=".",
    x_col="path",
    y_col="level",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

x_test = test_datagen.flow_from_dataframe(
    val_df,
    directory=".",
    x_col="path",
    y_col="level",
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Define the model
def build_model(input_shape, num_classes):
    in_lay = Input(input_shape)

    # Use InceptionV3 as the base model
    base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    # Extract features from the base model
    pt_features = base_model(in_lay)

    # Add Batch Normalization
    bn_features = BatchNormalization()(pt_features)

    # Attention mechanism
    attn_layer = Conv2D(64, kernel_size=(1, 1), padding='same', activation='relu')(bn_features)
    attn_layer = Conv2D(16, kernel_size=(1, 1), padding='same', activation='relu')(attn_layer)
    attn_layer = Conv2D(8, kernel_size=(1, 1), padding='same', activation='relu')(attn_layer)
    attn_layer = Conv2D(1, kernel_size=(1, 1), padding='valid', activation='sigmoid')(attn_layer)

    # Rescale attention weights
    up_c2_w = np.ones((1, 1, 1, 2048))
    up_c2 = Conv2D(2048, kernel_size=(1, 1), padding='same', activation='linear', use_bias=False, weights=[up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    # Apply attention to features
    mask_features = multiply([attn_layer, bn_features])

    # Global Average Pooling (GAP) instead of MaxPooling
    gap_features = GlobalAveragePooling2D()(mask_features)

    # Fully connected layers
    x = Dense(512, activation='relu')(gap_features)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layer
    out_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[in_lay], outputs=[out_layer])
    return model

In [ ]:
# Build the model
model = build_model(input_shape=(256, 256, 3), num_classes=x_train.num_classes)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Define callbacks
filepath = "best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, patience=15, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=6, min_lr=0.001, verbose=1)

In [ ]:
callbacks = [checkpoint, earlystop, reduce_lr]

In [ ]:
# Train the model
history = model.fit(
    x_train,
    steps_per_epoch=x_train.samples // 32,
    epochs=20,
    validation_data=x_test,
    validation_steps=x_test.samples // 32,
    callbacks=callbacks
)

In [ ]:
# Evaluate the model
train_loss, train_acc = model.evaluate(x_train)
test_loss, test_acc = model.evaluate(x_test)

print(f"Training Accuracy: {train_acc}")
print(f"Validation Accuracy: {test_acc}")

In [ ]:
# Confusion Matrix and Classification Report
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
class_labels = x_test.class_indices
class_labels = {v: k for k, v in class_labels.items()}

cm = confusion_matrix(x_test.classes, y_pred)
print('Confusion Matrix')
print(cm)

print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(x_test.classes, y_pred, target_names=target_names))

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 8))
plt.imshow(cm, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(target_names))
plt.xticks(tick_marks, target_names, rotation=90)
plt.yticks(tick_marks, target_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
model.save("improved_diabetic_retinopathy_model.h5")